# Tuần 3 bài tập Pandas: đọc ghi file csv,json, xlsx, xử lý dataframe (10 ví dụ khác biệt vs numpy) và vài ví dụ khác nữa.

In [1]:
import pandas as pd
import numpy as np

# ===============================
# 🔹 GIẢ LẬP DỮ LIỆU
# ===============================
np.random.seed(42)
dates = pd.date_range("2025-09-01", periods=10, freq="D")
products = ["A", "B", "C"]

# Tạo DataFrame doanh số
df = pd.DataFrame({
    "date": np.random.choice(dates, 20),
    "product": np.random.choice(products, 20),
    "qty": np.random.randint(1, 10, 20),
    "price": np.random.uniform(5.0, 15.0, 20).round(2)
})
df["revenue"] = df["qty"] * df["price"]

print("📊 Dữ liệu giả lập:")
print(df.head(), "\n")

# Dataset chi phí đơn vị
cost = pd.DataFrame({
    "product": ["A","B","C"],
    "unit_cost": [6.0, 4.0, 8.0]
})

# Dataset order chứa list (dùng explode)
orders = pd.DataFrame({
    "order_id":[1,2,3],
    "items":[["A","B"], ["C","A","C"], ["B","B"]]
})

# ===============================
# 🔹 CÁC VÍ DỤ XỬ LÝ
# ===============================

# 1) GroupBy
print("1) Doanh thu theo sản phẩm:")
print(df.groupby("product")["revenue"].sum(), "\n")

# 2) Pivot table
print("2) Pivot table (date x product):")
print(df.pivot_table(index="date", columns="product", values="revenue", aggfunc="sum", fill_value=0), "\n")

# 3) Merge (thêm cột profit)
df2 = df.merge(cost, on="product", how="left")
df2["profit"] = df2["revenue"] - df2["qty"]*df2["unit_cost"]
print("3) Merge + Profit:")
print(df2.head(), "\n")

# 4) Explode
print("4) Explode orders:")
print(orders.explode("items"), "\n")

# 5) Missing values
df2.loc[0,"unit_cost"] = np.nan
print("5) Fill missing unit_cost bằng median:")
print(df2["unit_cost"].fillna(df2["unit_cost"].median()).head(), "\n")

# 6) Resample (theo tuần)
print("6) Resample doanh thu tuần:")
print(df.set_index("date").resample("W")["revenue"].sum(), "\n")

# 7) Rolling
print("7) Rolling mean revenue (cửa sổ 3 ngày):")
print(
    df.set_index("date")
      .sort_index()                         # thêm dòng này là hết lỗi
      ["revenue"]
      .rolling("3D", min_periods=1)
      .mean()
      .head(),
    "\n"
)


# 8) Cut & qcut
df["rev_bin"] = pd.cut(df["revenue"], bins=[0,20,50,100], labels=["low","mid","high"])
df["rev_q"] = pd.qcut(df["revenue"], q=3, labels=["Q1","Q2","Q3"])
print("8) Cut & qcut:")
print(df[["revenue","rev_bin","rev_q"]].head(), "\n")

# 9) nlargest
print("9) Top 5 giao dịch lớn nhất:")
print(df.nlargest(5,"revenue")[["date","product","revenue"]], "\n")

# 10) String ops
df["product_upper"] = df["product"].str.upper()
print("10) String ops:")
print(df[["product","product_upper"]].head())


📊 Dữ liệu giả lập:
        date product  qty  price  revenue
0 2025-09-07       A    5  11.63    58.15
1 2025-09-04       A    3   8.12    24.36
2 2025-09-08       B    7  10.20    71.40
3 2025-09-05       B    5  10.47    52.35
4 2025-09-07       A    9   6.85    61.65 

1) Doanh thu theo sản phẩm:
product
A    306.84
B    358.57
C    344.26
Name: revenue, dtype: float64 

2) Pivot table (date x product):
product          A       B       C
date                              
2025-09-02    0.00    0.00   38.75
2025-09-03   25.50   21.80    0.00
2025-09-04   24.36  127.98    0.00
2025-09-05    0.00  123.47   21.96
2025-09-06   34.28    0.00   57.75
2025-09-07  119.80    0.00   57.56
2025-09-08    0.00   85.32  168.24
2025-09-10  102.90    0.00    0.00 

3) Merge + Profit:
        date product  qty  price  revenue  unit_cost  profit
0 2025-09-07       A    5  11.63    58.15        6.0   28.15
1 2025-09-04       A    3   8.12    24.36        6.0    6.36
2 2025-09-08       B    7  10.20    